In [73]:
import pandas as pd
import numpy as np
import sdv
from sdv.single_table.ctgan import CTGAN
from sdv.metadata import SingleTableMetadata
import random
import string

In [74]:
client = "national_scleroderma_foundation"
path = "/home/RMittal@ccsfundraising.com/ccs_predictive_modeling"
filename =  "%s_constituent_ccsdb.csv" %(client)
file = "%s/%s" %(path, filename)
df_cd = pd.read_csv(file, encoding="ISO-8859-1")
df_cd = df_cd.dropna(how="all", axis=1)
df_cd = df_cd.rename(columns={"solicit_codes": "solicit_codes_binary"})
df_cd["solicit_codes_binary"] = df_cd["solicit_codes_binary"].astype("bool")
df_cd = df_cd.drop(columns=["Unnamed: 0"])
df_cd["spouse_age"] = pd.to_datetime(df_cd["spouse_age"])

/tmp/ipykernel_1680355/1776846470.py:5: DtypeWarning: Columns (6,19,22,24,26,27,29,31,32,35,39,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cd = pd.read_csv(file, encoding="ISO-8859-1")


In [ ]:
df_cd_syn = df_cd.copy()
columns_nans ={key: df_cd_syn[key].isna().sum() for key in df_cd_syn.columns}

In [ ]:
df_cd_syn[df_cd_syn.select_dtypes(include='object').columns] = \
                        df_cd_syn.select_dtypes(include='object')\
                                 .apply(lambda x: x.str.replace("scleroderma", "Company", case=False))

In [ ]:
def create_array_with_nans(numbers, nans):
    # Combine the numbers and NaNs
    combined_list = numbers + [np.nan] * nans
    
    # Shuffle the combined list randomly
    np.random.shuffle(combined_list)
    
    # Convert the list to a numpy array
    result_array = np.array(combined_list)
    
    return result_array

# Example usage
# N1 = 5
# N2 = 3
# numbers_or_strings = [1, 2, "Alice", "Bob", 3.14]
# result = create_array_with_nans(numbers_or_strings, N2)

In [ ]:
# Generate a list of 100 random strings
def generate_random_strings(num_strings, string_length):
    characters = string.ascii_letters + string.digits
    return [''.join(random.choice(characters) for _ in range(string_length)) for _ in range(num_strings)]

In [ ]:
# Addresses

columns_address = ["address_1", "business_address", "seasonal_address"]

for col in columns_address:

    N = len(df_cd_syn) - columns_nans[col]

    # Generate a list of N random integers
    syn_house_numbers = [random.randint(1, N) for _ in range(N)]

    # Example: Generate a list of N random strings, each with a length of 10 characters
    syn_street_names = generate_random_strings(num_strings=N, string_length=10)

    # List of street suffixes
    suffixes = ["Ave", "Str", "Rd"]
    syn_suffix = [random.choice(suffixes) for _ in range(N)]

    col_array = [f"{num} {name} {suffix}" for num, name, suffix \
                        in zip(syn_house_numbers, syn_street_names, syn_suffix)]
    df_cd_syn[col] = create_array_with_nans(col_array, columns_nans[col])
    df_cd_syn[col] = df_cd_syn[col].astype(str)

In [ ]:
# Address_2

df_cd_syn["address_2"] = df_cd["address_2"].str.replace("apt", "Ste", case=False)
df_cd_syn["address_2"] = df_cd["address_2"].str.replace("ste", "Apt", case=False)

In [ ]:
# Emails

columns_email = ["personal_email", "spouse_email"]

for col in columns_email:

    N = len(df_cd_syn) - columns_nans[col]

    # Example: Generate a list of N random strings, each with a length of 5 characters
    syn_first_string = generate_random_strings(num_strings=N, string_length=5)

    # Example: Generate a list of N random strings, each with a length of 5 characters
    syn_second_string = generate_random_strings(num_strings=N, string_length=5)

    # List of street suffixes
    suffixes = ["@gmail.com", "@yahoo.com", "@verizon.net", "@outlook.com"]
    syn_suffix = [random.choice(suffixes) for _ in range(N)]

    col_array = [f"{first}.{second}{suffix}" for first, second, suffix \
                        in zip(syn_first_string, syn_second_string, syn_suffix)]
    df_cd_syn[col] = create_array_with_nans(col_array, columns_nans[col])
    df_cd_syn[col] = df_cd_syn[col].astype(str)

In [ ]:
# All Other Fields

columns_to_synthesize_string = ["first_name", "middle_name", "last_name", "spouse_name", "spouse_business_name"]
for col in columns_to_synthesize_string:

    N = len(df_cd_syn) - columns_nans[col]
    col_array = generate_random_strings(num_strings=N, string_length=10)
    
    df_cd_syn[col] = create_array_with_nans(col_array, columns_nans[col])
    df_cd_syn[col] = df_cd_syn[col].astype(str)

columns_to_synthesize_integer = ["constituent_id", "spouse_id", "home_phone", "cell_phone", "business_phone"]
for col in columns_to_synthesize_integer:

    N = len(df_cd) - columns_nans[col]
    
    #if col=="constituent_id":
        #print(N, columns_nans[col])
    
    if "phone" in col:
        col_array = [random.randint(1000000000, 9999999999) for _ in range(N)]
    else:
        col_array = random.sample(range(1, N + 1), N)
    
    df_cd_syn[col] = create_array_with_nans(col_array, columns_nans[col])
    df_cd_syn[col] = df_cd_syn[col].astype(str)

In [ ]:
df_cd_syn["is_deceased"] = df_cd_syn["is_deceased"].astype(bool)
df_cd_syn["is_individual"] = df_cd_syn["is_individual"].astype(bool)
df_cd_syn["loyalty"] = np.random.rand(len(df_cd_syn))

In [27]:
df_cd_syn.to_csv("synthetic_constituent_data.csv")